In [6]:
import requests
import pandas as pd
import numpy as np

invited_students = pd.read_csv(
    'GitHub Octernship Event Registration (Responses) - Invited Students.csv')
confirmed_students = pd.read_csv(
    'Confirmation_ GitHub Octernship Event Registration (Responses) - Form responses 1.csv')
df = pd.read_excel(
    "GitHub Octernship Event Registration (Responses).xlsx", sheet_name="Form responses 1")
all_responses = pd.read_csv(
    'GitHub Octernship Event Registration (Responses) - Form responses 1.csv'
)

Check for students that have filled the confirmation form.

In [7]:
unconfirmed_students = []
for student in invited_students['Email address']:
    if student not in list(confirmed_students['Email address']):
        unconfirmed_students.append(student)

print(f'Selected: {len(invited_students)}')
print(f"Confirmed: {len(confirmed_students)}")
print(f'Unconfirmed: {len(unconfirmed_students)}')

Selected: 162
Confirmed: 107
Unconfirmed: 58


In [8]:
def get_gender(row):
    name = row['First name']
    name = name.strip().split()[0]
    gender = requests.get(
        f'https://api.genderize.io?name={name}').json()["gender"]
    try:
        return gender.title()
    except AttributeError:
        return gender


In [19]:
df['gender'] = df.apply(get_gender, axis=1)
    
df['gender'].value_counts(normalize=True)

ConnectTimeout: HTTPSConnectionPool(host='api.genderize.io', port=443): Max retries exceeded with url: /?name=Peace (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DA5CA02FE0>, 'Connection to api.genderize.io timed out. (connect timeout=None)'))

Check for students that haven't been invited yet

In [9]:
students_not_invited = list(all_responses['Email address'])
for student in invited_students['Email address']:
    if student in students_not_invited:
        students_not_invited.remove(student)

print(f'Invited: {len(invited_students)}')
print(f"Not Invited: {len(students_not_invited)}")
print(f'All Responses: {len(all_responses)}')

Invited: 162
Not Invited: 314
All Responses: 475


In [10]:
students_not_invited_df = all_responses.loc[all_responses['Email address'].isin(
    students_not_invited)]
students_not_invited_df.to_csv('Virtual Invitation.csv', index=False)

Check for unregistered students who filled the Confirmation Form

In [11]:
unregistered_students = []
for student in confirmed_students['Email address']:
    if student not in list(all_responses['Email address']):
        unregistered_students.append(student)

print(f'Did not register: {len(unregistered_students)}')
unregistered_df = confirmed_students.loc[confirmed_students['Email address'].isin(
    unregistered_students)]

Did not register: 4


Check if anybody has the name in the main form

In [17]:
last_names = unregistered_df["Last name"].str.strip().tolist()
len(all_responses[all_responses["Last name"].isin(last_names)])

2

Update confirmed list

In [13]:
def get_tech_fields(email):
    response = all_responses.loc[all_responses["Email address"] == email,
                                 "Which of these tech fields do you have experience in?"]
    return response.values[0] if len(response) > 0 else ""


def get_uni(email):
    response = all_responses.loc[all_responses["Email address"] == email,
                                 "What is your university/institution?"]
    return response.values[0] if len(response) > 0 else ""


In [14]:
confirmed_students["Stack(s)"] = confirmed_students["Email address"].apply(
    get_tech_fields)
confirmed_students = confirmed_students[confirmed_students["Will you be physically present at the event?"] == "Yes"]
confirmed_students = confirmed_students.loc[
    :,
    ["Email address", "First name", "Last name", "Gender", "Stack(s)"]
]

In [15]:
confirmed_students["Institution"] = confirmed_students["Email address"].apply(
    get_uni)
confirmed_students.to_csv("updated_list.csv", index=False)

Get Supplementary List

In [16]:
sup_df = all_responses[all_responses["Email address"].isin(unconfirmed_students)].loc[:, [
    "Last name", "First name", "Email address"]]
sup_df["Institution"] = sup_df["Email address"].apply(
    get_uni)
sup_df["Stack(s)"] = sup_df["Email address"].apply(
    get_tech_fields)
sup_df.to_csv("supplementary_list.csv", index=False)